## run every time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/ml4nlp2/final_folder")
!ls

from  IPython. display  import  clear_output
import warnings
warnings. filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
baselines			   large_train_data_task1.csv
converted_og			   models_zoo.xlsx
data_process_code		   og_data
dev_data_task1.csv		   optuna_models
dev_data_task1_en.csv		   output_hard1.json
dev_data_task1_es.csv		   output_soft1.json
dev_data_task2.csv		   rohit_final_code
dev_data_task2_en.csv		   train_data_task1.csv
dev_data_task2_es.csv		   train_data_task1_en.csv
dev_data_task3.csv		   train_data_task1_es.csv
dev_data_task3_en.csv		   train_data_task2.csv
dev_data_task3_es.csv		   train_data_task2_en.csv
exist2023evaluation.py		   train_data_task2_es.csv
EXIST2023_test_clean.json	   train_data_task3.csv
golds				   train_data_task3_en.csv
good_paper_distilled_learning.pdf  train_data_task3_es.csv
hp_optune			   translated_og


In [ ]:
#set device
import torch
# device = torch.device("cpu")
device  =  torch. device('cuda:0'  if  torch. cuda. is_available() else  'cpu')
print(f"device is {device}")

device is cuda:0


In [ ]:
!pip install optuna
!pip install datasets
!pip install transformers==4.28.0
!pip install sentencepiece==0.1.98
!pip install pysentimiento
# !pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import optuna 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)

In [ ]:
import pandas as pd
import numpy as np
import random
import  matplotlib. pyplot  as  plt
from tqdm import tqdm

import torch
import torch.optim as optim
import  torch. nn. functional  as  F

from  IPython. display  import  clear_output
import warnings
warnings. filterwarnings('ignore')

#setting device agnostic version
# device = "cpu"
device  =  torch. device('cuda:0'  if  torch. cuda. is_available() else  'cpu')
print(f"computation will run on {device} now")

computation will run on cuda:0 now


In [ ]:
import re
def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

## construct optuna - all tasks hard

### construct optuna - task1 hard

In [ ]:
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

#instantiate label encoders
from sklearn.preprocessing import LabelEncoder
task1_encoder = LabelEncoder()
task2_encoder = LabelEncoder()
task3_encoder = LabelEncoder() 

def task1_hard_encode(df):
    task1_encoder.fit(all_task1_hard_labels)
    df['hard_label'] = task1_encoder.transform(df['hard_label'])
    return df

def task1_hard_decode(df):
    df["hard_label"] = task1_encoder.inverse_transform(df["hard_label"])
    return df

def task2_hard_encode(df):
    task2_encoder.fit(all_task2_hard_labels)
    df['hard_label'] = task1_encoder.transform(df['hard_label'])
    return df

def task2_hard_decode(df):
    df["hard_label"] = task2_encoder.inverse_transform(df["hard_label"])
    return df


import optuna 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset


#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task1.csv")
og_dev1 = pd.read_csv("dev_data_task1.csv")

all_task1_hard_labels = pd.concat([og_train1["hard_label"],og_dev1["hard_label"]])

train1_df = task1_hard_encode(og_train1)
# print(train1_df.columns)
train1_df = train1_df[["tweet","hard_label"]].dropna()
train1_df = train1_df[train1_df['hard_label'] != 2]
train1_df["tweet"] = simple_preprocess(train1_df["tweet"])

test1_df = task1_hard_encode(og_dev1)
test1_df = test1_df[["tweet","hard_label"]].dropna()
test1_df = test1_df[test1_df['hard_label'] != 2]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)


import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, test1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,ignore_mismatched_sizes=True)

# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/ml4nlp2/final_folder/optuna_models/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task1_hard'
MAX_LENGTH = 128

def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})



def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task1-hard', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

train1 (6064, 2)
<bound method NDFrame.head of                                                   tweet  hard_label
0     @USER Ignora al otro, es un capullo.El problem...           1
1     @USER Si comicsgate se parece en algo a gamerg...           0
2     @USER Lee sobre Gamergate, y como eso ha cambi...           0
4     @USER @USER @USER Entonces como así es el merc...           1
5     @USER Aaah sí. Andrew Dobson. El que se dedicó...           0
...                                                 ...         ...
6915  idk why y’all bitches think having half your a...           1
6916  This has been a part of an experiment with @US...           1
6917  "Take me already" "Not yet. You gotta be ready...           1
6918    @USER why do you look like a whore? /lh HTTPURL           1
6919  ik when mandy says “you look like a whore” i l...           1

[6064 rows x 2 columns]>
test1 (934, 2)
<bound method NDFrame.head of                                                   tweet  hard_labe

Some weights of the model checkpoint at sdadas/xlm-roberta-large-twitter were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dens

Step,Training Loss
500,0.493300
1000,0.381800
1500,0.275600
2000,0.160800


[I 2023-05-13 22:31:54,388] Trial 0 finished with value: 0.3187674554189046 and parameters: {'learning_rate': 3.395067563962819e-05, 'weight_decay': 0.004891290652279793, 'num_train_epochs': 3}. Best is trial 0 with value: 0.3187674554189046.


In [ ]:
print_custom('Finding study best parameters')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print_custom('Extract best study params')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print_custom('Create dictionary of the best hyperparameters')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}



Finding study best parameters
----------------------------------------------------------------------------------------------------


Extract best study params
----------------------------------------------------------------------------------------------------
The best learning rate is: 3.395067563962819e-05
The best weight decay is: 0.004891290652279793
The best epoch is : 3


Create dictionary of the best hyperparameters
----------------------------------------------------------------------------------------------------


### task 2 hard

In [ ]:
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

def tweet_preprocessor_spanish(text):
    """
    imported from pysentimiento 
    does the following: 
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="es", shorten=3)

def tweet_preprocessor_english(text):
    """
    imported from pysentimiento 
    does the following: 
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="en", shorten=3)

#instantiate label encoders
from sklearn.preprocessing import LabelEncoder
task_encoder = LabelEncoder()

def task_hard_encode(df):
    task_encoder.fit(all_task_hard_labels)
    df['hard_label'] = task_encoder.transform(df['hard_label'])
    return df

def task_hard_decode(df):
    df["hard_label"] = task_encoder.inverse_transform(df["hard_label"])
    return df


import optuna 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset


#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task2.csv")
og_dev1 = pd.read_csv("dev_data_task2.csv")

all_task_hard_labels = pd.concat([og_train1["hard_label"],og_dev1["hard_label"]])

train1_df = task_hard_encode(og_train1)
# print(train1_df.columns)
train1_df = train1_df[["tweet","hard_label"]].dropna()
train1_df = train1_df[train1_df['hard_label'] != 4] #removing nans
train1_df["tweet"] = simple_preprocess(train1_df["tweet"])

test1_df = task_hard_encode(og_dev1)
test1_df = test1_df[["tweet","hard_label"]].dropna()
test1_df = test1_df[test1_df['hard_label'] != 4] #removing nans
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)


import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, test1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4,ignore_mismatched_sizes=True)

# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/ml4nlp2/final_folder/optuna_models/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task1_hard'
MAX_LENGTH = 128

def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})



def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task1-hard', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

print_custom('Finding study best parameters')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print_custom('Extract best study params')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print_custom('Create dictionary of the best hyperparameters')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}

train1 (5496, 2)
<bound method NDFrame.head of                                                   tweet  hard_label
0     @USER Ignora al otro, es un capullo.El problem...           3
1     @USER Si comicsgate se parece en algo a gamerg...           2
2     @USER Lee sobre Gamergate, y como eso ha cambi...           2
5     @USER Aaah sí. Andrew Dobson. El que se dedicó...           2
7     @USER Esta gringa sigue llorando por el gamerg...           0
...                                                 ...         ...
6913  @USER Ma'am if I say that you look like a whor...           0
6915  idk why y’all bitches think having half your a...           0
6916  This has been a part of an experiment with @US...           1
6917  "Take me already" "Not yet. You gotta be ready...           0
6918    @USER why do you look like a whore? /lh HTTPURL           0

[5496 rows x 2 columns]>
test1 (841, 2)
<bound method NDFrame.head of                                                   tweet  hard_labe

Some weights of the model checkpoint at sdadas/xlm-roberta-large-twitter were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.de

Step,Training Loss
500,1.133600
1000,1.103600
1500,1.075400
2000,1.077800
2500,1.062900


[I 2023-05-13 23:24:57,734] Trial 0 finished with value: 1.089178811864522 and parameters: {'learning_rate': 0.0004005073373276802, 'weight_decay': 0.0053619187660176775, 'num_train_epochs': 4}. Best is trial 0 with value: 1.089178811864522.




Finding study best parameters
----------------------------------------------------------------------------------------------------


Extract best study params
----------------------------------------------------------------------------------------------------
The best learning rate is: 0.0004005073373276802
The best weight decay is: 0.0053619187660176775
The best epoch is : 4


Create dictionary of the best hyperparameters
----------------------------------------------------------------------------------------------------


### task 3 hard

In [ ]:
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

def tweet_preprocessor_spanish(text):
    """
    imported from pysentimiento 
    does the following: 
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="es", shorten=3)

def tweet_preprocessor_english(text):
    """
    imported from pysentimiento 
    does the following: 
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="en", shorten=3)

from sklearn.preprocessing import MultiLabelBinarizer

# Instantiate multi-label encoder
task_encoder = MultiLabelBinarizer()

   
def task_hard_encode(df):
    import numpy as np
    # Create a new DataFrame to store the transformed labels
    transformed_df = df.copy()
    task_encoder.fit(all_task_hard_labels)
    all_labels = []
    for i in range(len(df['hard_label'])):
        # Apply eval to each element of the label column
        all_labels.append(eval(df['hard_label'][i]))
    
    transformed_labels = task_encoder.transform(all_labels)
    # print(pd.DataFrame(transformed_labels))
    # transformed_df["hard_label"] = pd.DataFrame(transformed_labels)
    transformed_labels = pd.Series([x.tolist() for x in transformed_labels])
    transformed_df["hard_label"] = transformed_labels
    return transformed_df

def task_hard_decode(df):
    # Inverse transform the binary-encoded vectors back to original labels
    import numpy as np
    decoded_df = df.copy()
    decode_labels = np.array(decoded_df["hard_label"].tolist())

    # Inverse transform the label column using the provided task_encoder
    decoded_labels = task_encoder.inverse_transform(decode_labels)
    
    # Assign the decoded labels back to the DataFrame
    decoded_df["hard_labels"] = decoded_labels

    return decoded_df

import optuna 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset

#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task3.csv")
og_dev1 = pd.read_csv("dev_data_task3.csv")

train1_df = og_train1[["tweet","hard_label"]].dropna().reset_index(drop=True)

test1_df = og_dev1[["tweet","hard_label"]].dropna().reset_index(drop=True)

all_task_hard_labels = pd.concat([og_train1["hard_label"],og_dev1["hard_label"]]).dropna()
all_task_hard_labels = all_task_hard_labels.apply(lambda x: eval(x)).tolist()
print(all_task_hard_labels[0])

train1_df = task_hard_encode(train1_df)
# print(train1_df.columns)
train1_df = train1_df[["tweet","hard_label"]].dropna()
train1_df["tweet"] = simple_preprocess(train1_df["tweet"])

test1_df = task_hard_encode(test1_df)
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)


import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, test1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6,ignore_mismatched_sizes=True)

# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/ml4nlp2/final_folder/optuna_models/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task1_hard'
MAX_LENGTH = 128

def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})



def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task1-hard', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

print_custom('Finding study best parameters')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print_custom('Extract best study params')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print_custom('Create dictionary of the best hyperparameters')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}

['OBJECTIFICATION', 'SEXUAL-VIOLENCE']
train1 (6050, 2)
<bound method NDFrame.head of                                                   tweet          hard_label
0     @USER Ignora al otro, es un capullo.El problem...  [0, 0, 0, 1, 1, 0]
1     @USER Si comicsgate se parece en algo a gamerg...  [0, 0, 1, 0, 0, 0]
2     @USER Lee sobre Gamergate, y como eso ha cambi...  [0, 0, 1, 0, 0, 0]
3     @USER @USER @USER Entonces como así es el merc...  [1, 0, 0, 1, 0, 1]
4     @USER Aaah sí. Andrew Dobson. El que se dedicó...  [0, 0, 1, 0, 0, 0]
...                                                 ...                 ...
6045  idk why y’all bitches think having half your a...  [0, 1, 0, 1, 1, 1]
6046  This has been a part of an experiment with @US...  [0, 0, 0, 1, 0, 0]
6047  "Take me already" "Not yet. You gotta be ready...  [0, 0, 0, 0, 1, 0]
6048    @USER why do you look like a whore? /lh HTTPURL  [0, 1, 0, 1, 1, 1]
6049  ik when mandy says “you look like a whore” i l...  [0, 0, 0, 1, 0, 1]

[

Some weights of the model checkpoint at sdadas/xlm-roberta-large-twitter were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.de

Step,Training Loss
500,0.379900
1000,0.285000


[I 2023-05-13 23:46:13,792] Trial 0 finished with value: 0.308423602478552 and parameters: {'learning_rate': 1.5592115324779907e-05, 'weight_decay': 0.00019014604610473898, 'num_train_epochs': 2}. Best is trial 0 with value: 0.308423602478552.




Finding study best parameters
----------------------------------------------------------------------------------------------------


Extract best study params
----------------------------------------------------------------------------------------------------
The best learning rate is: 1.5592115324779907e-05
The best weight decay is: 0.00019014604610473898
The best epoch is : 2


Create dictionary of the best hyperparameters
----------------------------------------------------------------------------------------------------


## construct optuna all tasks soft

### task1 soft

In [ ]:
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

def tweet_preprocessor_spanish(text):
    """
    imported from pysentimiento 
    does the following: 
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="es", shorten=3)

def tweet_preprocessor_english(text):
    """
    imported from pysentimiento 
    does the following: 
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="en", shorten=3)

from sklearn.preprocessing import MultiLabelBinarizer

# Instantiate multi-label encoder
  
def convert_logits_to_list(logits_dict):
    logits_dict = eval(logits_dict)
    logits_list = [logits_dict["YES"], logits_dict["NO"]]
    return logits_list

def convert_list_to_logits(logits_list):
    logits_dict = {"YES": logits_list[0], "NO": logits_list[1]}
    return logits_dict

def check_dtype(given_data):
    return eval(given_data)

import optuna 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset

#load given data
#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task1.csv")
og_dev1 = pd.read_csv("dev_data_task1.csv")

og_train1["soft_label"] = og_train1["soft_label"].apply(convert_logits_to_list)
og_train1["tweet"] = simple_preprocess(og_train1["tweet"])
# og_train1["soft_label"] = og_train1["soft_label"].apply(check_dtype) 
train1_df = og_train1[["tweet","soft_label"]].dropna()


og_dev1["soft_label"] = og_dev1["soft_label"].apply(convert_logits_to_list)
og_dev1["tweet"] = simple_preprocess(og_dev1["tweet"])
# og_dev1["soft_label"] = og_dev1["soft_label"].apply(check_dtype) 
test1_df = og_dev1[["tweet","soft_label"]].dropna()


import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, test1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,ignore_mismatched_sizes=True)

# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/ml4nlp2/final_folder/optuna_models/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task1_hard'
MAX_LENGTH = 128

import torch.nn as nn
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["soft_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset


# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})

def custom_loss_fn(logits, soft_labels):
    probs = F.softmax(logits, dim=1)
    # Apply nn.CrossEntropyLoss
    loss = nn.CrossEntropyLoss(reduction="sum",label_smoothing=0.15)(probs, soft_labels)
    return loss


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task1-hard', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

print_custom('Finding study best parameters')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print_custom('Extract best study params')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print_custom('Create dictionary of the best hyperparameters')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}

Some weights of the model checkpoint at sdadas/xlm-roberta-large-twitter were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.de

Step,Training Loss
500,4.967700
1000,4.688100
1500,4.635300
2000,4.488400


[I 2023-05-14 09:12:07,328] Trial 0 finished with value: 4.657868658477937 and parameters: {'learning_rate': 2.5174100002380197e-05, 'weight_decay': 0.009582405316762398, 'num_train_epochs': 3}. Best is trial 0 with value: 4.657868658477937.




Finding study best parameters
----------------------------------------------------------------------------------------------------


Extract best study params
----------------------------------------------------------------------------------------------------
The best learning rate is: 2.5174100002380197e-05
The best weight decay is: 0.009582405316762398
The best epoch is : 3


Create dictionary of the best hyperparameters
----------------------------------------------------------------------------------------------------


### task2 soft

In [ ]:
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

def tweet_preprocessor_spanish(text):
    """
    imported from pysentimiento 
    does the following: 
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="es", shorten=3)

def tweet_preprocessor_english(text):
    """
    imported from pysentimiento 
    does the following: 
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="en", shorten=3)

from sklearn.preprocessing import MultiLabelBinarizer

def convert_logits_to_list(logits_dict):
    """
    order:
    1. classes are 
    REPORTED, NO, DIRECT, JUDGEMENTAL 
    """
    logits_dict = eval(logits_dict)
    logits_list = [logits_dict["REPORTED"], logits_dict["NO"],logits_dict["DIRECT"],logits_dict["JUDGEMENTAL"]]
    return logits_list

def convert_list_to_logits(logits_list):
    logits_dict = {"REPORTED": logits_list[0], "NO": logits_list[1],"DIRECT": logits_list[2], "JUDGEMENTAL": logits_list[3]}
    return logits_dict

def check_dtype(given_data):
    return eval(given_data)

import optuna 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset

#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task2.csv")
og_dev1 = pd.read_csv("dev_data_task2.csv")

og_train1["soft_label"] = og_train1["soft_label"].apply(convert_logits_to_list)
og_train1["tweet"] = simple_preprocess(og_train1["tweet"])
# og_train1["soft_label"] = og_train1["soft_label"].apply(check_dtype) 
train1_df = og_train1[["tweet","soft_label"]].dropna()


og_dev1["soft_label"] = og_dev1["soft_label"].apply(convert_logits_to_list)
og_dev1["tweet"] = simple_preprocess(og_dev1["tweet"])
# og_dev1["soft_label"] = og_dev1["soft_label"].apply(check_dtype) 
test1_df = og_dev1[["tweet","soft_label"]].dropna()

# print("train1",train1_df.shape)
print(train1_df.head)
# print("test1",test1_df.shape)
print(test1_df.head)

import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, test1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4,ignore_mismatched_sizes=True)

# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/ml4nlp2/final_folder/optuna_models/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task1_hard'
MAX_LENGTH = 128

import torch.nn as nn
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["soft_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset


# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})

def custom_loss_fn(logits, soft_labels):
    probs = F.softmax(logits, dim=1)
    # Apply nn.CrossEntropyLoss
    loss = nn.CrossEntropyLoss(reduction="sum",label_smoothing=0.15)(probs, soft_labels)
    return loss


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task1-hard', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

print_custom('Finding study best parameters')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print_custom('Extract best study params')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print_custom('Create dictionary of the best hyperparameters')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}

<bound method NDFrame.head of                                                   tweet  \
0     @USER Ignora al otro, es un capullo.El problem...   
1     @USER Si comicsgate se parece en algo a gamerg...   
2     @USER Lee sobre Gamergate, y como eso ha cambi...   
3     @USER Un retraso social bastante lamentable, g...   
4     @USER @USER @USER Entonces como así es el merc...   
...                                                 ...   
6915  idk why y’all bitches think having half your a...   
6916  This has been a part of an experiment with @US...   
6917  "Take me already" "Not yet. You gotta be ready...   
6918    @USER why do you look like a whore? /lh HTTPURL   
6919  ik when mandy says “you look like a whore” i l...   

                                             soft_label  
0     [0.5, 0.16666666666666602, 0.0, 0.333333333333...  
1     [0.0, 0.833333333333333, 0.16666666666666602, ...  
2                                  [0.0, 1.0, 0.0, 0.0]  
3     [0.33333333333333304, 0

Some weights of the model checkpoint at sdadas/xlm-roberta-large-twitter were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.de

Step,Training Loss
500,9.987300
1000,9.866800
1500,9.969400


[I 2023-05-14 09:35:26,908] Trial 0 finished with value: 9.94545673006144 and parameters: {'learning_rate': 0.0006429848369437259, 'weight_decay': 6.208951123230285e-05, 'num_train_epochs': 2}. Best is trial 0 with value: 9.94545673006144.




Finding study best parameters
----------------------------------------------------------------------------------------------------


Extract best study params
----------------------------------------------------------------------------------------------------
The best learning rate is: 0.0006429848369437259
The best weight decay is: 6.208951123230285e-05
The best epoch is : 2


Create dictionary of the best hyperparameters
----------------------------------------------------------------------------------------------------


### task 3 soft

In [ ]:
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

def tweet_preprocessor_spanish(text):
    """
    imported from pysentimiento 
    does the following: 
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="es", shorten=3)

def tweet_preprocessor_english(text):
    """
    imported from pysentimiento 
    does the following: 
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="en", shorten=3)

from sklearn.preprocessing import MultiLabelBinarizer

def convert_logits_to_list(logits_dict):
    """
    order:
    1. classes are 
    OBJECTIFICATION,'SEXUAL-VIOLENCE',NO,STEREOTYPING-DOMINANCE,IDEOLOGICAL-INEQUALITY,MISOGYNY-NON-SEXUAL-VIOLENCE
    """
    logits_dict = eval(logits_dict)
    logits_list = [logits_dict["OBJECTIFICATION"], logits_dict["SEXUAL-VIOLENCE"],logits_dict["NO"],logits_dict["STEREOTYPING-DOMINANCE"],logits_dict["IDEOLOGICAL-INEQUALITY"], logits_dict["MISOGYNY-NON-SEXUAL-VIOLENCE"]]
    return logits_list

def convert_list_to_logits(logits_list):
    logits_dict = {"OBJECTIFICATION": logits_list[0], "SEXUAL-VIOLENCE": logits_list[1],"NO": logits_list[2], "STEREOTYPING-DOMINANCE": logits_list[3],"IDEOLOGICAL-INEQUALITY": logits_list[4], "MISOGYNY-NON-SEXUAL-VIOLENCE": logits_list[5]}
    return logits_dict

def check_dtype(given_data):
    return eval(given_data)


import optuna 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset

#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task3.csv")
og_dev1 = pd.read_csv("dev_data_task3.csv")

og_train1["soft_label"] = og_train1["soft_label"].apply(convert_logits_to_list)
og_train1["tweet"] = simple_preprocess(og_train1["tweet"])
# og_train1["soft_label"] = og_train1["soft_label"].apply(check_dtype) 
train1_df = og_train1[["tweet","soft_label"]].dropna()


og_dev1["soft_label"] = og_dev1["soft_label"].apply(convert_logits_to_list)
og_dev1["tweet"] = simple_preprocess(og_dev1["tweet"])
# og_dev1["soft_label"] = og_dev1["soft_label"].apply(check_dtype) 
test1_df = og_dev1[["tweet","soft_label"]].dropna()

# print("train1",train1_df.shape)
print(train1_df.head)
# print("test1",test1_df.shape)
print(test1_df.head)

import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, test1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6,ignore_mismatched_sizes=True)

# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/ml4nlp2/final_folder/optuna_models/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task1_hard'
MAX_LENGTH = 128

import torch.nn as nn
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["soft_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset


# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})

def custom_loss_fn(logits, soft_labels):
    # Convert probabilities to logits using the inverse sigmoid (logit) function
    # eps = 1e-7  # Small constant to avoid numerical instability
    # logits = torch.log(probabilities.clamp(min=eps, max=1-eps) / (1 - probabilities.clamp(min=eps, max=1-eps)))
    loss = nn.BCEWithLogitsLoss(reduction='sum')(logits, soft_labels)
    return loss


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task1-hard', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

print_custom('Finding study best parameters')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print_custom('Extract best study params')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print_custom('Create dictionary of the best hyperparameters')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}

<bound method NDFrame.head of                                                   tweet  \
0     @USER Ignora al otro, es un capullo.El problem...   
1     @USER Si comicsgate se parece en algo a gamerg...   
2     @USER Lee sobre Gamergate, y como eso ha cambi...   
3     @USER Un retraso social bastante lamentable, g...   
4     @USER @USER @USER Entonces como así es el merc...   
...                                                 ...   
6915  idk why y’all bitches think having half your a...   
6916  This has been a part of an experiment with @US...   
6917  "Take me already" "Not yet. You gotta be ready...   
6918    @USER why do you look like a whore? /lh HTTPURL   
6919  ik when mandy says “you look like a whore” i l...   

                                             soft_label  
0     [0.33333333333333304, 0.33333333333333304, 0.1...  
1     [0.16666666666666602, 0.0, 0.833333333333333, ...  
2                        [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]  
3     [0.16666666666666602, 0

Some weights of the model checkpoint at sdadas/xlm-roberta-large-twitter were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.de

Step,Training Loss
500,21.516400
1000,18.906800
1500,18.440300


[I 2023-05-14 10:11:07,560] Trial 0 finished with value: 19.555706311709915 and parameters: {'learning_rate': 1.121348453013019e-06, 'weight_decay': 0.0005161669504916949, 'num_train_epochs': 2}. Best is trial 0 with value: 19.555706311709915.




Finding study best parameters
----------------------------------------------------------------------------------------------------


Extract best study params
----------------------------------------------------------------------------------------------------
The best learning rate is: 1.121348453013019e-06
The best weight decay is: 0.0005161669504916949
The best epoch is : 2


Create dictionary of the best hyperparameters
----------------------------------------------------------------------------------------------------


In [ ]:
# with a different custom loss function
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

def tweet_preprocessor_spanish(text):
    """
    imported from pysentimiento 
    does the following: 
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="es", shorten=3)

def tweet_preprocessor_english(text):
    """
    imported from pysentimiento 
    does the following: 
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="en", shorten=3)

from sklearn.preprocessing import MultiLabelBinarizer

def convert_logits_to_list(logits_dict):
    """
    order:
    1. classes are 
    OBJECTIFICATION,'SEXUAL-VIOLENCE',NO,STEREOTYPING-DOMINANCE,IDEOLOGICAL-INEQUALITY,MISOGYNY-NON-SEXUAL-VIOLENCE
    """
    logits_dict = eval(logits_dict)
    logits_list = [logits_dict["OBJECTIFICATION"], logits_dict["SEXUAL-VIOLENCE"],logits_dict["NO"],logits_dict["STEREOTYPING-DOMINANCE"],logits_dict["IDEOLOGICAL-INEQUALITY"], logits_dict["MISOGYNY-NON-SEXUAL-VIOLENCE"]]
    return logits_list

def convert_list_to_logits(logits_list):
    logits_dict = {"OBJECTIFICATION": logits_list[0], "SEXUAL-VIOLENCE": logits_list[1],"NO": logits_list[2], "STEREOTYPING-DOMINANCE": logits_list[3],"IDEOLOGICAL-INEQUALITY": logits_list[4], "MISOGYNY-NON-SEXUAL-VIOLENCE": logits_list[5]}
    return logits_dict

def check_dtype(given_data):
    return eval(given_data)


import optuna 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset

#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task3.csv")
og_dev1 = pd.read_csv("dev_data_task3.csv")

og_train1["soft_label"] = og_train1["soft_label"].apply(convert_logits_to_list)
og_train1["tweet"] = simple_preprocess(og_train1["tweet"])
# og_train1["soft_label"] = og_train1["soft_label"].apply(check_dtype) 
train1_df = og_train1[["tweet","soft_label"]].dropna()


og_dev1["soft_label"] = og_dev1["soft_label"].apply(convert_logits_to_list)
og_dev1["tweet"] = simple_preprocess(og_dev1["tweet"])
# og_dev1["soft_label"] = og_dev1["soft_label"].apply(check_dtype) 
test1_df = og_dev1[["tweet","soft_label"]].dropna()

# print("train1",train1_df.shape)
print(train1_df.head)
# print("test1",test1_df.shape)
print(test1_df.head)

import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, test1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6,ignore_mismatched_sizes=True)

# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 16
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = '/content/drive/MyDrive/ml4nlp2/final_folder/optuna_models/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task1_hard'
MAX_LENGTH = 128

import torch.nn as nn
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["soft_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset


# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})

#with cross entropy loss and label smoothing
def custom_loss_fn(logits, soft_labels):
    probs = torch.sigmoid(logits)
    # Apply nn.CrossEntropyLoss
    loss = nn.CrossEntropyLoss(reduction="sum",label_smoothing=0.15)(probs, soft_labels)
    return loss


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=False
    )

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

study = optuna.create_study(study_name='hp-search-task1-hard', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

print_custom('Finding study best parameters')
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print_custom('Extract best study params')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print_custom('Create dictionary of the best hyperparameters')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}

<bound method NDFrame.head of                                                   tweet  \
0     @USER Ignora al otro, es un capullo.El problem...   
1     @USER Si comicsgate se parece en algo a gamerg...   
2     @USER Lee sobre Gamergate, y como eso ha cambi...   
3     @USER Un retraso social bastante lamentable, g...   
4     @USER @USER @USER Entonces como así es el merc...   
...                                                 ...   
6915  idk why y’all bitches think having half your a...   
6916  This has been a part of an experiment with @US...   
6917  "Take me already" "Not yet. You gotta be ready...   
6918    @USER why do you look like a whore? /lh HTTPURL   
6919  ik when mandy says “you look like a whore” i l...   

                                             soft_label  
0     [0.33333333333333304, 0.33333333333333304, 0.1...  
1     [0.16666666666666602, 0.0, 0.833333333333333, ...  
2                        [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]  
3     [0.16666666666666602, 0

Some weights of the model checkpoint at sdadas/xlm-roberta-large-twitter were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.de

Step,Training Loss
500,14.640100
1000,13.983600
1500,14.043500
2000,13.821900


[I 2023-05-14 10:45:06,110] Trial 0 finished with value: 14.076958482189594 and parameters: {'learning_rate': 8.036768439133452e-06, 'weight_decay': 8.398333136282262e-05, 'num_train_epochs': 3}. Best is trial 0 with value: 14.076958482189594.




Finding study best parameters
----------------------------------------------------------------------------------------------------


Extract best study params
----------------------------------------------------------------------------------------------------
The best learning rate is: 8.036768439133452e-06
The best weight decay is: 8.398333136282262e-05
The best epoch is : 3


Create dictionary of the best hyperparameters
----------------------------------------------------------------------------------------------------
